In [ ]:
! ls *.gv

In [ ]:
! pip install pydot

In [ ]:
import os
from collections import defaultdict
from subprocess import getstatusoutput

import pydot

try:
    from tqdm.auto import tqdm
except (ModuleNotFoundError, ImportError) as e:
    print(f"Could not import tqdm: {str(e)}")
    def tqdm(collection, *args, **kwags):
        return iter(collection)

outdir = "unrolled"
os.makedirs(outdir, exist_ok=True)

In [ ]:
dotfile_path = "design.gv"
g = pydot.graph_from_dot_file(dotfile_path)
assert len(g) == 1
g = g[0]

ignore_base = set('/"\'\\n\t\n\r ')
ignore_set = {"graph", "node", "\n"}

all_nodes = {}
all_edges = {}
unroll = defaultdict(set)
todo = [g]
while todo:
    current = todo.pop()
    for edge in current.get_edges():
        src = edge.get_source()
        dst = edge.get_destination()
        if not isinstance(src, str) or not isinstance(dst, str):
            continue
        elif any([
            src in ignore_set,
            set(src) <= ignore_base,
            dst in ignore_set,
            set(dst) <= ignore_base]):
            continue
            
        all_edges[(src, dst)] = edge
        unroll[src].add((src, dst))
        unroll[dst].add((src, dst))
        
    for node in current.get_nodes():
        name = node.get_name()
        if set(name) <= ignore_base:
            node.set("peripheries", 0)
            continue
        elif name in ignore_set:
            continue
        if node.get("peripheries") is None:
            node.set("peripheries", 1)
        node.set("URL", f"{name}.html")
        
        if name == "graph":
            print("!!!")
        all_nodes[name] = node

    todo.extend(current.get_subgraphs())

In [ ]:
errors = []
for node, edges in tqdm(list(unroll.items())):
    gg = pydot.Graph()
    added_nodes = set()
    added_edges = set()
    
    for src, dst in edges:
        if src not in added_nodes:
            gg.add_node(all_nodes[src])
            added_nodes.add(src)
        if dst not in added_nodes:
            gg.add_node(all_nodes[dst])
            added_nodes.add(dst)
        if (src, dst) not in added_edges:
            gg.add_edge(all_edges[(src, dst)])
            added_edges.add((src, dst))
    
    gvfile = os.path.join(outdir, f"{node}.gv")
    open(gvfile, "w").write(gg.to_string())
    
    svgfile = os.path.join(outdir, f"{node}.svg")
    status, output = getstatusoutput(f"dot -Tsvg {gvfile}")
    if status == 0:
        svg = output[output.index("<svg"):].replace('xlink:href="graph.html"', "")    
        open(svgfile, "w").write(svg)
        htmlfile = os.path.join(outdir, f"{node}.html")
        open(htmlfile, "w").write(f"""<!doctype html>
<head><title>{node}</title></head>
<body>
<center><a href="index.html"><h3>Back to diagram</h3></a><br/></center><hr/>
{svg}
</body>
</html>
""")
    else:
        errors.append((node, gvfile, status, output))

In [ ]:
index_gv = os.path.join(outdir, "index.gv")
open(index_gv, "w").write(g.to_string())

index_svg = os.path.join(outdir, "index.svg")
status, output = getstatusoutput(f"dot -Tsvg {index_gv}")
assert status == 0, output
svg = output[output.index("<svg"):].replace('xlink:href="graph.html"', "")
open(index_svg, "w").write(svg)

index_html = os.path.join(outdir, "index.html")
open(index_html, "w").write(f"""<!doctype html>
<head><title>{dotfile_path}</title></head>
<body>
<center><a href="index.html"><h3>Back to diagram</h3></a><br/></center><hr/>
{svg}
</body>
</html>
""")